# Data Processing

In [14]:
import pandas as pd
import os
import json
from IPython.display import display
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql import SparkSession

clean_path="../data/clean/"
processing_path="../data/processing/"

# Initial Data
_parameters = pd.read_json(clean_path+"parameters.json")
_countries = pd.read_json(clean_path+"countries.json")
_measuraments = pd.read_json(clean_path+"measurements.json")
display(_parameters)
display(_countries[_countries['name']=='Spain'])
display(_measuraments)
print(_countries[_countries['name']=='Spain'].parameters.values[0])

,displayName,name,preferredUnit
1,PM10,pm10,µg/m³
2,PM2.5,pm25,µg/m³
3,O₃ mass,o3,µg/m³
4,CO mass,co,µg/m³
5,NO₂ mass,no2,µg/m³
6,SO₂ mass,so2,µg/m³
7,NO₂,no2,ppm
8,CO,co,ppm
9,SO₂,so2,ppm
10,O₃,o3,ppm


,cities,code,name,parameters
38,59,ES,Spain,"[co, no2, o3, pm1, pm10, pm25, so2, um010, um0..."


,average,day,parameter,unit
0,52.0092,2020-12-31,o3,µg/m³
1,11.5678,2020-12-31,pm10,µg/m³
2,14.7604,2020-12-31,no2,µg/m³
3,3.4066,2020-12-31,so2,µg/m³
4,322.8080,2020-12-31,co,µg/m³
...,...,...,...,...
2177,3.1374,2020-01-01,so2,µg/m³
2178,25.6587,2020-01-01,pm10,µg/m³
2179,412.7953,2020-01-01,co,µg/m³
2180,22.7752,2020-01-01,no2,µg/m³


['co', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'so2', 'um010', 'um025', 'um100']


In [2]:
# Create Spark Context
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.getOrCreate()

In [3]:
measuraments= spark.read.json(clean_path+"measurements.json").rdd
measuraments.first()

Row(average=52.0092, day='2020-12-31', parameter='o3', unit='µg/m³')

In [4]:
#measuraments = measuraments.map(lambda i: ((i['day'],i['parameter'],i['average'],i['unit'])))

In [6]:
co = measuraments.filter(lambda i: i['parameter'] == 'co')
no2 = measuraments.filter(lambda i: i['parameter'] == 'no2')
o3 = measuraments.filter(lambda i: i['parameter'] == 'o3')
so2 = measuraments.filter(lambda i: i['parameter'] == 'so2')